In [1]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from ucimlrepo import fetch_ucirepo 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# fetch dataset 
phishing_websites = fetch_ucirepo(id=327) 
  
# data (as pandas dataframes) 
X = phishing_websites.data.features 
y = phishing_websites.data.targets

# Chia tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")

Train size: (8844, 30), Test size: (2211, 30)


In [2]:

from scipy.stats import randint, uniform

param_distributions = {
    'n_estimators': randint(50, 300),
    'max_depth': randint(3, 15)
}


In [22]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score
from scipy.stats import randint, uniform

class RandomizedSearchCV_from_scratch:
    def __init__(self, estimator, param_distributions, n_iter=10, cv=5, random_state=None):
        self.estimator = estimator  # Mô hình cần tối ưu hóa
        self.param_distributions = param_distributions  # Phân phối tham số
        self.n_iter = n_iter  # Số lần thử nghiệm
        self.cv = cv  # Số fold cho cross-validation
        self.random_state = random_state  # Seed cho tái lập

        # Thiết lập seed nếu cần
        if self.random_state is not None:
            np.random.seed(self.random_state)

    def sample_params(self):
        """
        Lấy mẫu ngẫu nhiên các tham số từ phân phối đã cho.
        """
        sampled_params = {}
        for param, dist in self.param_distributions.items():
            sampled_params[param] = dist.rvs()
        return sampled_params

    def cross_val_score(self, X, y, params):
        """
        Thực hiện cross-validation (StratifiedKFold) để tính điểm cho mô hình với tham số đã cho.
        """
        skf = StratifiedKFold(n_splits=self.cv, shuffle=True, random_state=self.random_state)
        scores = []

        # Chạy CV
        for train_idx, valid_idx in skf.split(X, y):
            _X_train, _X_valid = X.iloc[train_idx], X.iloc[valid_idx]
            _y_train, _y_valid = y.iloc[train_idx], y.iloc[valid_idx]

            # Huấn luyện mô hình với tham số đã chọn
            self.estimator.set_params(**params)
            self.estimator.fit(_X_train, _y_train)

            # Dự đoán và tính điểm accuracy
            y_pred = self.estimator.predict(_X_valid)
            score = accuracy_score(_y_valid, y_pred)
            scores.append(score)

        return np.mean(scores), np.std(scores)  # Trả về điểm số trung bình của tất cả các fold

    def fit(self, X, y):
        """
        Tiến hành tìm kiếm ngẫu nhiên các tham số và huấn luyện mô hình với Cross-Validation.
        """
        best_score = -np.inf
        best_params = None
        results = []

        for i in range(self.n_iter):
            # Lấy mẫu các tham số
            sampled_params = self.sample_params()

            # Tính điểm cho tham số hiện tại với Cross-Validation
            mean_score, std_score = self.cross_val_score(X, y, sampled_params)

            results.append({'params': sampled_params, 'mean_score': mean_score, 'std_score': std_score})

            # Cập nhật nếu điểm số hiện tại là tốt nhất
            if mean_score > best_score:
                best_score = mean_score
                best_params = sampled_params

        # Lưu kết quả và trả về tham số tốt nhất
        self.results_ = results
        self.best_score_ = best_score
        self.best_params_ = best_params

In [23]:
rf = RandomForestClassifier(random_state=42)

In [ ]:
rf = RandomForestClassifier(random_state=42)
search = RandomizedSearchCV_from_scratch(rf, param_distributions, n_iter=10, cv=5, random_state=42)
search.fit(X_train, y_train)

C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed whe

In [25]:
results = search.results_
results

[{'params': {'n_estimators': 152, 'max_depth': 6},
  'mean_score': np.float64(0.9337398228413425),
  'std_score': np.float64(0.005562064595556827)},
 {'params': {'n_estimators': 142, 'max_depth': 13},
  'mean_score': np.float64(0.9614432445152692),
  'std_score': np.float64(0.0032797031601484255)},
 {'params': {'n_estimators': 121, 'max_depth': 7},
  'mean_score': np.float64(0.9362274235258307),
  'std_score': np.float64(0.0032954692818318917)},
 {'params': {'n_estimators': 152, 'max_depth': 12},
  'mean_score': np.float64(0.9601992843056255),
  'std_score': np.float64(0.003000547059202927)},
 {'params': {'n_estimators': 260, 'max_depth': 9},
  'mean_score': np.float64(0.946969937255243),
  'std_score': np.float64(0.003280785232637579)},
 {'params': {'n_estimators': 124, 'max_depth': 13},
  'mean_score': np.float64(0.9608778254964202),
  'std_score': np.float64(0.002995073924076509)},
 {'params': {'n_estimators': 137, 'max_depth': 7},
  'mean_score': np.float64(0.9369059007696656),
  '

In [26]:
results = search.results_
# Tạo DataFrame từ list các kết quả
import pandas as pd
df = pd.DataFrame(results)
params_df = pd.DataFrame(df['params'].tolist())
final_df = pd.concat([params_df, df[['mean_score','std_score']]], axis=1).sort_values(by='mean_score', ascending=False)
final_df.reset_index(drop=True, inplace=True)

In [27]:
print(final_df)

   n_estimators  max_depth  mean_score  std_score
0           142         13    0.961443   0.003280
1           124         13    0.960878   0.002995
2           152         12    0.960199   0.003001
3           149         10    0.951493   0.004522
4           260          9    0.946970   0.003281
5           199          7    0.937811   0.003379
6           137          7    0.936906   0.003160
7           121          7    0.936227   0.003295
8           152          6    0.933740   0.005562
9           201          5    0.930687   0.004489


In [41]:
model = RandomForestClassifier(random_state=42)
model.set_params(**search.best_params_)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Test accuracy", accuracy_score(y_test, y_pred))

C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Test accuracy 0.9638172772501131


In [38]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
np.random.seed(43)
clf = RandomizedSearchCV(rf, param_distributions=param_distributions, random_state=42,  scoring='accuracy',cv=cv_strategy, n_iter=10)
sklearn_rdsearch = clf.fit(X_train, y_train)
print(sklearn_rdsearch.best_params_)

C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed whe

{'max_depth': 13, 'n_estimators': 137}


In [39]:
print(pd.DataFrame(sklearn_rdsearch.cv_results_))

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.511050      0.008687         0.032515        0.000248   
1       0.303015      0.005141         0.021679        0.000967   
2       0.303799      0.005077         0.019813        0.000200   
3       0.636774      0.033421         0.041870        0.002223   
4       0.279744      0.015025         0.018960        0.000812   
5       0.342094      0.007243         0.023883        0.000302   
6       0.295213      0.001761         0.020076        0.000452   
7       0.507180      0.041229         0.032529        0.003728   
8       0.362046      0.019583         0.021956        0.001620   
9       0.103837      0.002530         0.008355        0.000659   

   param_max_depth  param_n_estimators  \
0                9                 229   
1               13                 121   
2                7                 152   
3               12                 260   
4                9                 124   
5         

In [42]:
rf = RandomForestClassifier(random_state=42)
rf.set_params(**sklearn_rdsearch.best_params_)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print("Test accuracy", accuracy_score(y_test, y_pred))

C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Test accuracy 0.9633649932157394


## Kết hợp RandomizedSearch và GridSearch

In [43]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# Bước 1: Randomized Search với không gian tham số rộng
param_distributions = {
    'n_estimators': randint(50, 300),
    'max_depth': randint(3, 15)
}

In [49]:
rf = RandomForestClassifier(random_state=42)
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
random_search = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions=param_distributions,
    n_iter=10,
    cv=cv_strategy,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)
print("Best parameters from Randomized Search:", random_search.best_params_)
print("Best_score",random_search.best_score_)


C:\Users\ADMIN\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best parameters from Randomized Search: {'max_depth': 13, 'n_estimators': 137}
Best_score 0.9613301223433235


In [ ]:
fine_tune_params = {
    'n_estimators': np.arange(
        max(10, best_params['n_estimators'] - 20),  # Giảm 20 nhưng không nhỏ hơn 10
        best_params['n_estimators'] + 21,  # Tăng 20
        step=5  # Khoảng cách giữa các giá trị
    ).tolist(),
    
    'max_depth': np.arange(
        max(1, best_params['max_depth'] - 3),  # Giảm tối đa 3
        best_params['max_depth'] + 4,  # Tăng tối đa 3
        step=1
    ).tolist()}